In [81]:
#import libraries
import pymongo
import json
from pymongo import MongoClient
import pandas as pd
import numpy as np
import pymysql
import os

conda install -c anaconda mysql-python
conda install -c anaconda mysql-connector-python
pip install MySQL-python
pip install MySQL-python-connector

In [82]:
#Set nutrition level tags to read (for reference only)
nutriTags = (
"energy_100g",
"proteins_100g",
"carbohydrates_100g",
"sugars_100g",
"fat_100g",
"saturated-fat_100g",
"trans-fat_100g",
"cholesterol_100g",
"fiber_100g",
"sodium_100g",
"vitamin-a_100g",
"vitamin-d_100g",
"vitamin-c_100g"
)

In [83]:
#Set product level tags to read (for reference only)
prodTags = (
"product_name",
"code",
"_id",
"brands"
)

### Connect to MongoDB

In [84]:
#connect to local database server
client = MongoClient()

#switch to test DB
db = client.food

## Functions

In [85]:
# function to print only first n documents (to avoid perf/memory issues)
def printhead(cursor, n):
    for idx,document in enumerate(cursor):
        if idx <= n: 
            print(document)
        else:
            break

In [86]:
# function to check if value exists in dataframe for nutrition info
def readNutrition(df, key):
    try:   
        myValue = df["nutriments"]
        myValue = df["nutriments"][key]       
    except:
        myValue = "NA"
    return(myValue)

In [87]:
# function to check if value exists in object
def readValue(df, key):
    try:   
        myValue = df[key]
    except:
        myValue = "NA"
    return(myValue)

In [88]:
# function to read all files in folder for UPCs
def readFiles(path):
    files = []
    print("Reading UPC files from: ",path)
    for filePath in sorted(os.listdir(path)):
        print("Reading file: ",filePath)
        fullPath = os.path.join(path, filePath)
        files.append(fullPath)
    print("Read ",len(files)," files")
    return(files)

### Read input file with UPCs to search in OpenFoodFacts

In [135]:
#Paths for load files
myPath = "D:/IRIDataCopy/UPC/"
myPanelistSalesPath = "D:/IRIDataCopy/Year12/PANEL/" #Not used
mySQLUploadPath = "C:\\\\ProgramData\\\\MySQL\\\\MySQL Server 8.0\\\\Uploads\\\\"

## Read all files in path

In [12]:
all_files = readFiles(myPath)

Reading UPC files from:  D:/IRIDataCopy/UPC/
Reading file:  Cereal.txt
Reading file:  Coffee.txt
Reading file:  HotDog.txt
Reading file:  MargButter.txt
Reading file:  Mayo.txt
Reading file:  Meat.txt
Reading file:  Milk.txt
Reading file:  MustKetch.txt
Reading file:  PeanutButter.txt
Reading file:  Pizza.txt
Reading file:  SaltySnacks.txt
Reading file:  Soup.txt
Reading file:  SpagSauce.txt
Reading file:  SugarSub.txt
Reading file:  Yogurt.txt
Read  15  files


## Search UPCs in OpenFoodFacts

In [13]:
%%time

myprods = db.products

upcMatched = []
tCount=0
tAllCount=0

for filePath in all_files:
    print("Searching UPCs from: ",filePath)
    file = open(filePath, "r")
    all_upcs = file.read().splitlines()
    iCount = 0
    iAllCount = 0
    for upc in all_upcs:
        result = pd.DataFrame({"Category":os.path.splitext(os.path.basename(file.name))[0], "Product":list(myprods.find({"code" : "0"+ upc }).limit(1))})
        if not result.empty:
            upcMatched.append(result)
            #Update counters
            iCount = iCount + 1
            tCount = tCount + 1
        iAllCount = iAllCount + 1
        tAllCount = tAllCount + 1
    print("Matched ", str(iCount), " UPCs out of ",str(iAllCount)," in: ",filePath)

print("Total matched ", str(tCount), " UPCs out of ",str(tAllCount)," in all files.")

Searching UPCs from:  D:/IRIDataCopy/UPC/Cereal.txt
Matched  790  UPCs out of  7512  in:  D:/IRIDataCopy/UPC/Cereal.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/Coffee.txt
Matched  164  UPCs out of  13429  in:  D:/IRIDataCopy/UPC/Coffee.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/HotDog.txt
Matched  159  UPCs out of  2215  in:  D:/IRIDataCopy/UPC/HotDog.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/MargButter.txt
Matched  114  UPCs out of  631  in:  D:/IRIDataCopy/UPC/MargButter.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/Mayo.txt
Matched  168  UPCs out of  1246  in:  D:/IRIDataCopy/UPC/Mayo.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/Meat.txt
Matched  714  UPCs out of  8808  in:  D:/IRIDataCopy/UPC/Meat.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/Milk.txt
Matched  740  UPCs out of  9802  in:  D:/IRIDataCopy/UPC/Milk.txt
Searching UPCs from:  D:/IRIDataCopy/UPC/MustKetch.txt
Matched  320  UPCs out of  2248  in:  D:/IRIDataCopy/UPC/MustKetch.txt
Searching UPCs from:  D:/IRIDataCopy/U

## Prepare data for insertion

In [73]:
%%time

myNutriDataFrame = pd.DataFrame()

for i, product in enumerate(upcMatched):
    headerVal_Category = product["Category"].values
    headerVal_UPCCode = readValue(product["Product"][0],"code")
    headerVal_brand = readValue(product["Product"][0],"brands")
    headerVal_name = readValue(product["Product"][0],"product_name")
    #Read nutritional tags
    nutriVal_energy = readNutrition(product["Product"][0],"energy_100g")    
    nutriVal_protein = readNutrition(product["Product"][0],"proteins_100g")    
    nutriVal_carbs = readNutrition(product["Product"][0],"carbohydrates_100g")    
    nutriVal_sugars = readNutrition(product["Product"][0],"sugars_100g")    
    nutriVal_fat = readNutrition(product["Product"][0],"fat_100g")    
    nutriVal_saturatedfat = readNutrition(product["Product"][0],"saturated-fat_100g")    
    nutriVal_transfat = readNutrition(product["Product"][0],"trans-fat_100g")    
    nutriVal_cholestrol = readNutrition(product["Product"][0],"cholesterol_100g")    
    nutriVal_fiber = readNutrition(product["Product"][0],"fiber_100g")    
    nutriVal_sodium = readNutrition(product["Product"][0],"sodium_100g")    
    nutriVal_vita = readNutrition(product["Product"][0],"vitamin-a_100g")    
    nutriVal_vitd = readNutrition(product["Product"][0],"vitamin-d_100g")    
    nutriVal_vitc = readNutrition(product["Product"][0],"vitamin-c_100g")    
    myNutriDataFrame = myNutriDataFrame.append({
        "Category" :headerVal_Category, 
        "UPCCode" :headerVal_UPCCode, 
        "UPCKey" :"",
        "Brand" :headerVal_brand,
        "Name" :headerVal_name ,
        "energy_100g" :nutriVal_energy,
        "proteins_100g":nutriVal_protein,
        "carbohydrates_100g":nutriVal_carbs,
        "sugars_100g":nutriVal_sugars,
        "fat_100g":nutriVal_fat,
        "saturated-fat_100g":nutriVal_saturatedfat,
        "trans-fat_100g":nutriVal_transfat,
        "cholesterol_100g":nutriVal_cholestrol,
        "fiber_100g":nutriVal_fiber,
        "sodium_100g":nutriVal_sodium,
        "vitamin-a_100g":nutriVal_vita,
        "vitamin-d_100g":nutriVal_vitd,
        "vitamin-c_100g":nutriVal_vitc},
        ignore_index=True)
    

Wall time: 46 s


In [74]:
myNutriDataFrame.head(5)

,Brand,Category,Name,UPCCode,UPCKey,carbohydrates_100g,cholesterol_100g,energy_100g,fat_100g,fiber_100g,proteins_100g,saturated-fat_100g,sodium_100g,sugars_100g,trans-fat_100g,vitamin-a_100g,vitamin-c_100g,vitamin-d_100g
0,Quisling Media,[Cereal],"Laura's, Granola, Strawberry Vanilla",0689076109918,,64,0,1590,10,6,10,1,0.08,22,NA,0,0.0048,NA
1,Leila Bay Trading Co.,[Cereal],"Leila Bay Trading Co., Cranberry Almond Granola",0023882190034,,58.93,0,1866,21.43,5.4,7.14,8.04,0.179,23.21,0,0,0,NA
2,Ambrosial,[Cereal],"Athenian Harvest, Fiber Boost Goji Berry Hazel...",0689584000035,,70.91,0,1674,9.09,9.1,12.73,2.73,0.009,23.64,0,0.0002181,0,NA
3,NA,[Cereal],Original,0765670777194,,56,NA,2176,28,NA,14,7,0.00787402,20,NA,NA,NA,NA
4,Anderson Trail Inc.,[Cereal],Oatsnack,0854526001221,,56.67,0,1674,16.67,6.7,6.67,5,0.233,30,0,0,0.32,NA


In [78]:
myNutriDataFrame.drop_duplicates(subset="UPCCode", keep='first', inplace=True)
myNutriDataFrame.to_csv("D:\IRIDataCopy\Test\Output.csv", sep='|', index=False, encoding='utf-8')

for index, row in myNutriDataFrame.iterrows():
    print(row['Brand'],row['Category'],row['Name'],row['UPCCode'],row['UPCKey'],row['carbohydrates_100g'],row['cholesterol_100g'],row['energy_100g'],row['fat_100g'],row['fiber_100g'],row['proteins_100g'],row['saturated-fat_100g'],row['sodium_100g'],row['sugars_100g'],row['trans-fat_100g'],row['vitamin-a_100g'],row['vitamin-c_100g'],row['vitamin-d_100g'])    

In [140]:
all_files_sales = readFiles(mySQLUploadPath)

Reading UPC files from:  C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\
Reading file:  Delivery_Stores.DAT
Reading file:  Nutri_Output.csv
Reading file:  beer_PANEL_DK_1687_1739.DAT
Reading file:  beer_PANEL_GK_1687_1739.DAT
Reading file:  beer_PANEL_MK_1687_1739.DAT
Reading file:  blades_PANEL_DK_1687_1739.DAT
Reading file:  blades_PANEL_GK_1687_1739.DAT
Reading file:  blades_PANEL_MK_1687_1739.DAT
Reading file:  carbbev_PANEL_DK_1687_1739.DAT
Reading file:  carbbev_PANEL_GK_1687_1739.DAT
Reading file:  carbbev_PANEL_MK_1687_1739.DAT
Reading file:  cigets_PANEL_DK_1687_1739.DAT
Reading file:  cigets_PANEL_GK_1687_1739.DAT
Reading file:  cigets_PANEL_MK_1687_1739.DAT
Reading file:  coffee_PANEL_DK_1687_1739.DAT
Reading file:  coffee_PANEL_GK_1687_1739.DAT
Reading file:  coffee_PANEL_MK_1687_1739.DAT
Reading file:  coldcer_PANEL_DK_1687_1739.DAT
Reading file:  coldcer_PANEL_GK_1687_1739.DAT
Reading file:  coldcer_PANEL_MK_1687_1739.DAT
Reading file:  deod_PANEL_DK_1687_1739.DAT
Rea

In [143]:
%%time

# Open database connection
db = pymysql.connect("localhost","root","root","nutripipe" )


for file in all_files_sales:
    if "_PANEL_" in os.path.basename(file): 
        sqlhead = "LOAD DATA INFILE '"
        sqltail = "' INTO TABLE nutripipe.panelist_sales FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' IGNORE 1 LINES (panid, week, minute, units, outlet, dollars, iri_key, colupc);"
        execsql = sqlhead + file + sqltail
        
        # prepare a cursor object using cursor() method
        cursor = db.cursor() 
        try:
            # Execute the SQL command
            cursor.execute(execsql)
            db.commit()
        except:
            print ("Error: unable to load data for ", file)        
    else:
        continue

db.close()

C:\Users\sriva\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'colupc' at row 1")
  result = self._query(query)
C:\Users\sriva\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'colupc' at row 2")
  result = self._query(query)
C:\Users\sriva\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'colupc' at row 3")
  result = self._query(query)
C:\Users\sriva\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'colupc' at row 4")
  result = self._query(query)
C:\Users\sriva\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'colupc' at row 5")
  result = self._query(query)
C:\Users\sriva\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'colupc' at row 6")
  result = self._query(query)
C:\Users\sriva\Anaconda3\lib\site-packag

Wall time: 7.38 s
